In [236]:
import networkx as nx 
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd

In [237]:
coordinates_file = '../../Fieldwork/QGIS maps/SamplingSites.csv'
threshold = 185 # max distance is between HM-C and HM-D at ~ 180
threshold_times = 4

In [238]:
coord_df = pd.read_csv(coordinates_file)[['field_1','Easting','Northing']]
HM_coord_df = coord_df[coord_df['field_1'].str.contains("HM")]
HM_coord_array = HM_coord_df[['Easting','Northing']].to_numpy()
# HM_coord_array

In [239]:
HM_dist_matr = np.zeros((len(HM_coord_df),len(HM_coord_df)))
for i in range(len(HM_coord_df)):
    for j in range(len(HM_coord_df)):
        HM_dist_matr[i,j] = np.linalg.norm(HM_coord_array[i,:]-HM_coord_array[j,:])
pd.DataFrame(HM_dist_matr)

,0,1,2,3,4,5,6,7,8
0,0.000000,422.946460,316.191533,239.487669,171.843540,11.003556,154.497746,277.507299,397.120900
1,422.946460,0.000000,121.384265,221.261640,401.378646,429.621507,289.337223,209.127721,176.328978
2,316.191533,121.384265,0.000000,99.878320,279.997425,324.173050,205.709633,179.401727,220.435700
3,239.487669,221.261640,99.878320,0.000000,180.135097,248.999835,174.801352,212.110817,292.055969
4,171.843540,401.378646,279.997425,180.135097,0.000000,182.353455,241.623764,342.169288,447.779806
5,11.003556,429.621507,324.173050,248.999835,182.353455,0.000000,157.224327,279.893525,399.587674
6,154.497746,289.337223,205.709633,174.801352,241.623764,157.224327,0.000000,123.058499,242.624636
7,277.507299,209.127721,179.401727,212.110817,342.169288,279.893525,123.058499,0.000000,119.695520
8,397.120900,176.328978,220.435700,292.055969,447.779806,399.587674,242.624636,119.695520,0.000000


In [240]:
def get_trunc_matr(array,fill=np.nan,keep_diagonal=False):
    new_array = array.copy()
    j = 0
    if keep_diagonal: diag = 1
    else: diag = 0

    for i in range(len(new_array)-diag):
        new_array[(i+diag):,j] = fill
        j += 1
    return new_array

In [241]:
pd.DataFrame(HM_dist_matr)

,0,1,2,3,4,5,6,7,8
0,0.000000,422.946460,316.191533,239.487669,171.843540,11.003556,154.497746,277.507299,397.120900
1,422.946460,0.000000,121.384265,221.261640,401.378646,429.621507,289.337223,209.127721,176.328978
2,316.191533,121.384265,0.000000,99.878320,279.997425,324.173050,205.709633,179.401727,220.435700
3,239.487669,221.261640,99.878320,0.000000,180.135097,248.999835,174.801352,212.110817,292.055969
4,171.843540,401.378646,279.997425,180.135097,0.000000,182.353455,241.623764,342.169288,447.779806
5,11.003556,429.621507,324.173050,248.999835,182.353455,0.000000,157.224327,279.893525,399.587674
6,154.497746,289.337223,205.709633,174.801352,241.623764,157.224327,0.000000,123.058499,242.624636
7,277.507299,209.127721,179.401727,212.110817,342.169288,279.893525,123.058499,0.000000,119.695520
8,397.120900,176.328978,220.435700,292.055969,447.779806,399.587674,242.624636,119.695520,0.000000


In [242]:
HM_dist_matr_trunc = get_trunc_matr(HM_dist_matr,keep_diagonal=0,fill=0)
pd.DataFrame(HM_dist_matr_trunc)

,0,1,2,3,4,5,6,7,8
0,0.0,422.94646,316.191533,239.487669,171.843540,11.003556,154.497746,277.507299,397.120900
1,0.0,0.00000,121.384265,221.261640,401.378646,429.621507,289.337223,209.127721,176.328978
2,0.0,0.00000,0.000000,99.878320,279.997425,324.173050,205.709633,179.401727,220.435700
3,0.0,0.00000,0.000000,0.000000,180.135097,248.999835,174.801352,212.110817,292.055969
4,0.0,0.00000,0.000000,0.000000,0.000000,182.353455,241.623764,342.169288,447.779806
5,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,157.224327,279.893525,399.587674
6,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,123.058499,242.624636
7,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,119.695520
8,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [243]:
pd.DataFrame(HM_dist_matr)

,0,1,2,3,4,5,6,7,8
0,0.000000,422.946460,316.191533,239.487669,171.843540,11.003556,154.497746,277.507299,397.120900
1,422.946460,0.000000,121.384265,221.261640,401.378646,429.621507,289.337223,209.127721,176.328978
2,316.191533,121.384265,0.000000,99.878320,279.997425,324.173050,205.709633,179.401727,220.435700
3,239.487669,221.261640,99.878320,0.000000,180.135097,248.999835,174.801352,212.110817,292.055969
4,171.843540,401.378646,279.997425,180.135097,0.000000,182.353455,241.623764,342.169288,447.779806
5,11.003556,429.621507,324.173050,248.999835,182.353455,0.000000,157.224327,279.893525,399.587674
6,154.497746,289.337223,205.709633,174.801352,241.623764,157.224327,0.000000,123.058499,242.624636
7,277.507299,209.127721,179.401727,212.110817,342.169288,279.893525,123.058499,0.000000,119.695520
8,397.120900,176.328978,220.435700,292.055969,447.779806,399.587674,242.624636,119.695520,0.000000


In [244]:
def get_neighb_matr(array,threshold,threshold_times):
    array[np.where(array>threshold)] = threshold_times * threshold
    return array

HM_neighb_matr_trunc = get_neighb_matr(HM_dist_matr_trunc,threshold,threshold_times)
pd.DataFrame(HM_neighb_matr_trunc)

,0,1,2,3,4,5,6,7,8
0,0.0,740.0,740.000000,740.00000,171.843540,11.003556,154.497746,740.000000,740.000000
1,0.0,0.0,121.384265,740.00000,740.000000,740.000000,740.000000,740.000000,176.328978
2,0.0,0.0,0.000000,99.87832,740.000000,740.000000,740.000000,179.401727,740.000000
3,0.0,0.0,0.000000,0.00000,180.135097,740.000000,174.801352,740.000000,740.000000
4,0.0,0.0,0.000000,0.00000,0.000000,182.353455,740.000000,740.000000,740.000000
5,0.0,0.0,0.000000,0.00000,0.000000,0.000000,157.224327,740.000000,740.000000
6,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,123.058499,740.000000
7,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,119.695520
8,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


In [245]:
def PCoA(matrix):
    # https://lookmanolowo.web.illinois.edu/2024/03/13/principal-coordinate-analysis-hands-on/
    
    # square the matrix to emphasize the hidden structure within the dataset
    D = matrix.copy()
    D_2 = np.pow(D,2)

    n = len(D)

    # get centering matrix A
    J = np.ones_like(D)
    I = np.diag(np.diag(J))
    A = I - (J / n)

    # get W, which encodes the variance and structure of the original dataset as represented by the squared distances, after being adjusted by double-centering
    W = -.5 * np.linalg.multi_dot([A, D_2, A.T])
    
    # find the eigenvalues
    eigenvalues, eigenvectors = np.linalg.eigh(W)

    return eigenvalues, eigenvectors

In [246]:
a = np.array([
[0, 4.551, 18.204, 8.306, 14.041],
[4.551, 0, 12.714, 3.918, 12.551],
[18.204, 12.714, 0, 8.633, 8.735],
[8.306, 3.918, 8.633, 0, 7.490],
[14.041, 12.551, 8.735, 7.490, 0]])

eigenvalues, eigenvectors = PCoA(a)

In [250]:
pos_eigenvectors = eigenvectors
pos_eigenvectors[np.where(pos_eigenvectors<=0)] = 0
pos_eigenvalues = eigenvalues
pos_eigenvalues[np.where(pos_eigenvalues<=0)] = 0
pos_eigenvalues

array([  0.        ,   0.        ,   0.        ,  48.71634135,
       200.93066193])